 ### Generate training and testing data

In [1]:
#!/usr/bin/env python
# coding: utf-8
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential
from PIL import Image
import tensorflow as tf
import numpy as np
import pprint as pp

shape = ['triangle', 'square', 'circle', 'star']
inputArray = []
outputArray = []
testingArray = []
output = []
num = 3000

# Generate model training data
for i in range(num):
    oneHot = np.zeros(4)
    oneHot[i % 4] = 1
    imageFile = Image.open("./shapes/" + shape[i % 4] + "/" + str(i + 1) + ".png")
    inputArray.append((tf.keras.preprocessing.image.img_to_array(imageFile) / 255.0))
    outputArray.append(oneHot)

# Generate test data and null prediction value
for i in range(num + 50, num + 250):
    path = "./shapes/" + shape[i % 4] + "/" + str(i + 1) + ".png"
    output.append([path, None])
    imageFile = Image.open(path)
    testingArray.append((tf.keras.preprocessing.image.img_to_array(imageFile) / 255.0))

 ### Define neural network model

In [2]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (200, 200, 1), activation='relu'))
model.add(MaxPooling2D(pool_size = (8, 8)))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'sigmoid'))
model.add(Dense(units = 4, activation = 'sigmoid'))
model.summary()

W0720 17:21:31.636630  1060 deprecation.py:506] From c:\users\jaures\documents\workspace\sujunn\env\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2359424   
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 2,360,260
Trainable params: 2,360,260
Non-trainable params: 0
_________________________________________________________________


### Compile and fit model

In [3]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(np.array(inputArray), np.array(outputArray), batch_size = 32, epochs = 5, validation_split = 0.1, shuffle=True)

Train on 2700 samples, validate on 300 samples
Epoch 1/5
2700/2700 [==============================] - 32s 12ms/sample - loss: 0.0738 - acc: 0.8515 - val_loss: 0.0121 - val_acc: 0.9967
Epoch 2/5
2700/2700 [==============================] - 31s 11ms/sample - loss: 0.0073 - acc: 0.9974 - val_loss: 0.0049 - val_acc: 0.9933
Epoch 3/5
2700/2700 [==============================] - 31s 11ms/sample - loss: 0.0030 - acc: 0.9985 - val_loss: 0.0077 - val_acc: 0.9900
Epoch 4/5
2700/2700 [==============================] - 31s 11ms/sample - loss: 0.0017 - acc: 0.9993 - val_loss: 0.0030 - val_acc: 0.9933
Epoch 5/5
2700/2700 [==============================] - 31s 11ms/sample - loss: 0.0011 - acc: 0.9996 - val_loss: 0.0046 - val_acc: 0.9900


### Predict using finished model

In [4]:
prediction = model.predict(np.array(testingArray))

### Parse predictions

In [5]:
# Normalize prediction and associate best guess with category
for i in range(len(prediction)):
    prediction[i] /= sum(prediction[i])
    maxVal = 0
    for j in range(len(prediction[i])):
        if (prediction[i][j] > maxVal):
            maxVal = prediction[i][j]
            index = j
    output[i][1] = (shape[index])
    
pp.pprint(output)

[['./shapes/circle/3051.png', 'circle'],
 ['./shapes/star/3052.png', 'star'],
 ['./shapes/triangle/3053.png', 'triangle'],
 ['./shapes/square/3054.png', 'square'],
 ['./shapes/circle/3055.png', 'circle'],
 ['./shapes/star/3056.png', 'star'],
 ['./shapes/triangle/3057.png', 'triangle'],
 ['./shapes/square/3058.png', 'square'],
 ['./shapes/circle/3059.png', 'circle'],
 ['./shapes/star/3060.png', 'star'],
 ['./shapes/triangle/3061.png', 'triangle'],
 ['./shapes/square/3062.png', 'square'],
 ['./shapes/circle/3063.png', 'circle'],
 ['./shapes/star/3064.png', 'star'],
 ['./shapes/triangle/3065.png', 'triangle'],
 ['./shapes/square/3066.png', 'square'],
 ['./shapes/circle/3067.png', 'circle'],
 ['./shapes/star/3068.png', 'star'],
 ['./shapes/triangle/3069.png', 'triangle'],
 ['./shapes/square/3070.png', 'square'],
 ['./shapes/circle/3071.png', 'circle'],
 ['./shapes/star/3072.png', 'star'],
 ['./shapes/triangle/3073.png', 'triangle'],
 ['./shapes/square/3074.png', 'square'],
 ['./shapes/circ

In [6]:
m = np.array(Image.open('./shapes/validation/star_1.jpg').resize((200,200)).convert(mode="L"), dtype='float32')
m.shape

(200, 200)

In [7]:
m = m.reshape((1,200,200,1))
model.predict(m)

model.save('shaper.h5')

In [8]:
# Image.open("./shapes/validation/star_1.png").resize((200,200)).convert(mode='L').save('./shapes/validation/star_1.jpg')